In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from full_fred.fred import Fred
import yfinance as yf
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import datetime
from scipy.stats import norm
from datetime import datetime
import math

In [2]:
def daily(x):
    dta = yf.download(tickers=x,period='max',interval = '1d')
    dta[x] = dta['Close']
    dta = dta.drop(columns=['Open','High','Low','Close','Adj Close','Volume'])
    dta[x] = np.array(dta[x])
    return dta[x]

def data(x,y):
    a = fred.get_series_df(x,frequency=y)
    a.value = a.value.replace('.',np.nan)
    a.value = a.value.ffill()
    a.index = a.date
    a = a.drop(columns=['date','realtime_start','realtime_end'])
    a.value = a.value.astype('float')
    return a

def black_scholes(S, X, t, r, sigma, option_type='call'):
    """
    Calculate Black-Scholes option price.

    Parameters:
    S (float): Current stock price.
    X (float): Strike price.
    t (float): Time to expiration in years.
    r (float): Risk-free interest rate.
    sigma (float): Volatility of the stock.
    option_type (str): 'call' or 'put'.

    Returns:
    float: Black-Scholes option price.
    """
    # Convert inputs to floats if they are arrays
    if isinstance(S, (np.ndarray, list)):
        S = float(S[0])
    if isinstance(X, (np.ndarray, list)):
        X = float(X[0])
    if isinstance(t, (np.ndarray, list)):
        t = float(t[0])
    if isinstance(r, (np.ndarray, list)):
        r = float(r[0])
    if isinstance(sigma, (np.ndarray, list)):
        sigma = float(sigma[0])
    
    # Calculate d1 and d2
    d1 = (math.log(S / X) + (r + 0.5 * sigma ** 2) * t) / (sigma * math.sqrt(t))
    d2 = d1 - sigma * math.sqrt(t)
    
    if option_type == 'call':
        # Calculate call option price
        option_price = S * norm.cdf(d1) - X * math.exp(-r * t) * norm.cdf(d2)
    elif option_type == 'put':
        # Calculate put option price
        option_price = X * math.exp(-r * t) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Option type must be 'call' or 'put'")
    
    return option_price

In [3]:
fred = Fred('key.txt')
fred.set_api_key_file('key.txt')

True

For options analysis the 2 Year Treasury Yield used as RFR

In [4]:
rfr = data("DGS2",'d')

In [5]:
current_rfr = rfr.tail(1)['value'].values/100

In [6]:
def options_data(ticker,nth_expiration,type):
    
    x = yf.Ticker(ticker)
    
    option_type = ''
    if type =='call':
        option_type = 0
    elif type =='put':
        option_type = 1
    
    expirations = x.options
    expiration = x.options[nth_expiration]
    
    data = x.option_chain(expiration)
    data = data[option_type]
    
    stock = daily(ticker)
    stock_monthly = stock.groupby(pd.Grouper(freq='ME')).mean()
    stock_monthly_change = stock_monthly.pct_change().fillna(0)
    sigma = stock_monthly_change.std()
    current_price = stock.tail(1)
    expiration = pd.to_datetime(expiration)

    time_years = expiration - datetime.now()
    time_years = time_years.total_seconds() / (24*60*60)
    time_years = time_years/365.25   
    return data , expirations, sigma, current_price.values, time_years, expiration

def black_scholes(S, X, t, r, sigma, option_type='call'):
    """
    Calculate Black-Scholes option price.

    Parameters:
    S (float): Current stock price.
    X (float): Strike price.
    t (float): Time to expiration in years.
    r (float): Risk-free interest rate.
    sigma (float): Volatility of the stock.
    option_type (str): 'call' or 'put'.

    Returns:
    float: Black-Scholes option price.
    """
    # Calculate d1 and d2
    d1 = (math.log(S / X) + (r + 0.5 * sigma ** 2) * t) / (sigma * math.sqrt(t))
    d2 = d1 - sigma * math.sqrt(t)
    
    if option_type == 'call':
        # Calculate call option price
        option_price = S * norm.cdf(d1) - X * math.exp(-r * t) * norm.cdf(d2)
    elif option_type == 'put':
        # Calculate put option price
        option_price = X * math.exp(-r * t) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Option type must be 'call' or 'put'")
    
    return option_price

def calculate_greeks(S, X, t, r, sigma, option_type='call'):
    """
    Calculate Black-Scholes option Greeks.

    Parameters:
    S (float): Current stock price.
    X (float): Strike price.
    t (float): Time to expiration in years.
    r (float): Risk-free interest rate.
    sigma (float): Volatility of the stock.
    option_type (str): 'call' or 'put'.

    Returns:
    dict: A dictionary containing the option price and Greeks (Delta, Gamma, Theta, Vega, Rho).
    """
    S = float(S)
    X = float(X)
    t = float(t)
    r = float(r)
    sigma = float(sigma)
    
    # Calculate d1 and d2
    d1 = (math.log(S / X) + (r + 0.5 * sigma ** 2) * t) / (sigma * math.sqrt(t))
    d2 = d1 - sigma * math.sqrt(t)
    
    option_price = black_scholes(S, X, t, r, sigma, option_type)
    
    # Calculate Greeks
    delta = norm.cdf(d1) if option_type == 'call' else norm.cdf(d1) - 1
    gamma = norm.pdf(d1) / (S * sigma * math.sqrt(t))
    vega = S * norm.pdf(d1) * math.sqrt(t)
    theta = (-S * norm.pdf(d1) * sigma / (2 * math.sqrt(t)) 
             - r * X * math.exp(-r * t) * norm.cdf(d2 if option_type == 'call' else -d2))
    rho = X * t * math.exp(-r * t) * norm.cdf(d2 if option_type == 'call' else -d2)
    
    return {
        'option_price': option_price,
        'delta': delta,
        'gamma': gamma,
        'theta': theta,
        'vega': vega,
        'rho': rho
    }
    

In [7]:
spy = options_data("SPY",15,"call")

[*********************100%%**********************]  1 of 1 completed


In [8]:
black_scholes(float(spy[3]),spy[0][spy[0]['strike'] == 600]['strike'].values,spy[4],current_rfr,spy[0][spy[0]['strike'] == 600]['impliedVolatility'].values)

C:\Users\Yoga\AppData\Local\Temp\ipykernel_12644\3545342168.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  black_scholes(float(spy[3]),spy[0][spy[0]['strike'] == 600]['strike'].values,spy[4],current_rfr,spy[0][spy[0]['strike'] == 600]['impliedVolatility'].values)
C:\Users\Yoga\AppData\Local\Temp\ipykernel_12644\2209688951.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  d1 = (math.log(S / X) + (r + 0.5 * sigma ** 2) * t) / (sigma * math.sqrt(t))
C:\Users\Yoga\AppData\Local\Temp\ipykernel_12644\2209688951.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you

array([3.475779])

In [22]:
x = yf.Ticker("SPY")
x.options

('2024-07-01',
 '2024-07-02',
 '2024-07-03',
 '2024-07-05',
 '2024-07-12',
 '2024-07-19',
 '2024-07-26',
 '2024-07-31',
 '2024-08-02',
 '2024-08-16',
 '2024-08-30',
 '2024-09-20',
 '2024-09-30',
 '2024-10-18',
 '2024-10-31',
 '2024-11-15',
 '2024-11-29',
 '2024-12-20',
 '2024-12-31',
 '2025-01-17',
 '2025-01-31',
 '2025-03-21',
 '2025-03-31',
 '2025-06-20',
 '2025-09-19',
 '2025-12-19',
 '2026-01-16',
 '2026-06-18',
 '2026-12-18')

In [10]:
calculate_greeks(float(spy[3]),spy[0][spy[0]['strike'] == 650]['strike'].values,spy[4],current_rfr,spy[0][spy[0]['strike'] == 650]['impliedVolatility'].values)

C:\Users\Yoga\AppData\Local\Temp\ipykernel_12644\1455409644.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  calculate_greeks(float(spy[3]),spy[0][spy[0]['strike'] == 650]['strike'].values,spy[4],current_rfr,spy[0][spy[0]['strike'] == 650]['impliedVolatility'].values)
C:\Users\Yoga\AppData\Local\Temp\ipykernel_12644\2209688951.py:75: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  X = float(X)
C:\Users\Yoga\AppData\Local\Temp\ipykernel_12644\2209688951.py:77: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing t

{'option_price': 0.37816589297344905,
 'delta': 0.02397127899078064,
 'gamma': 0.0013083458872099971,
 'theta': -3.815314590457727,
 'vega': 18.87273933438805,
 'rho': 4.78575456964488}

In [ ]:
def options_data(ticker,type):
    x = yf.Ticker(ticker)
    option_type = ''
    if type =='call':
        option_type = 0
    elif type =='put':
        option_type = 1
        
    data = x.option_chain[option_type]
    
    

In [ ]:
def options_data(ticker,nth_expiration,type):
    
    x = yf.Ticker(ticker)
    
    option_type = ''
    if type =='call':
        option_type = 0
    elif type =='put':
        option_type = 1
    
    expirations = x.options
    expiration = x.options[nth_expiration]
    
    data = x.option_chain(expiration)
    data = data[option_type]
    
    stock = daily(ticker)
    stock_monthly = stock.groupby(pd.Grouper(freq='ME')).mean()
    stock_monthly_change = stock_monthly.pct_change().fillna(0)
    sigma = stock_monthly_change.std()
    current_price = stock.tail(1)
    expiration = pd.to_datetime(expiration)

    time_years = expiration - datetime.now()
    time_years = time_years.total_seconds() / (24*60*60)
    time_years = time_years/365.25   
    return data , expirations, sigma, current_price.values, time_years, expiration